In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

2024-04-08 10:46:17.390318: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-08 10:46:17.553352: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 10:46:18.363176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 10:46:18.363246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 10:46:18.546289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
tweets = pd.read_csv('../tweets/tweets_10k.csv')

tweets.head()

,target,id,date,flag,user,text
0,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there ...
1,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,So I had a dream last night. I remember a sig...
2,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,@girlyghost ohh poor sickly you (((hugs)) ho...
3,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again
4,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,@MissKeriBaby wish I was in LA right now


In [4]:
np.random.seed(100)

tweets = tweets.loc[np.random.choice(list(range(tweets.shape[0])), 1000, replace = False)]

In [5]:
t = TweetTokenizer()

def remove_handle(tweet):
    tokens = t.tokenize(tweet)

    if tokens[0].startswith('@'):
        return ' '.join(tokens[1:])

    return ' '.join(tokens)


tweets['cleaned'] = tweets['text'].apply(remove_handle)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(tweets['cleaned'], tweets['target'], test_size = 0.2, random_state = 1)

In [17]:
dbt = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = dbt(list(x_train), max_length = 30, truncation = True, padding = True)
test_encodings = dbt(list(x_test), max_length = 30, truncation = True, padding = True)

In [21]:
train = tf.data.Dataset.from_tensor_slices((dict(train_encodings), list(y_train))).batch(2)
test = tf.data.Dataset.from_tensor_slices((dict(test_encodings), list(y_test))).batch(2)

In [19]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [23]:
model.compile(optimizer = optimizers.Adam(learning_rate = 3e-5),
              loss = losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

model.fit(train, batch_size = 2, epochs = 2)

Epoch 1/2


400/400 [==============================] - 137s 292ms/step - loss: 0.5974 - accuracy: 0.6725
Epoch 2/2
400/400 [==============================] - 115s 286ms/step - loss: 0.3325 - accuracy: 0.8587


In [69]:
model.evaluate(test, return_dict = True, batch_size = 2)

100/100 [==============================] - 3s 30ms/step - loss: 0.2859 - accuracy: 0.9200


{'loss': 0.28591465950012207, 'accuracy': 0.9200000166893005}